In [1]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

from multiprocessing import Pool

import time
import pandas as pd

Все API-запросы к Тасс можно формировать следующим образом через обычный реквест: 

Если нужны вообще все новости:
'http://tass.ru/api/news/lenta?limit=50&before=1488624568'

Если нужны новости только из 25 раздела - экономика:
'http://tass.ru/api/news/lenta?limit=200&sections%5B%5D=25&unmarked[]=1581998&before=1488624568'

In [2]:
# функция, которая крутит вниз ленту из новостей ТАСС
def _lenta(before,limit=200):
    mainpage = 'http://tass.ru/api/news/lenta?limit='+str(limit)+'&before='+str(before)
    response = requests.get(mainpage)
    dic = response.json()
    df = dic['articles']
    w =[[item['title'],item['time'],item['url'],item['is_breaking_news'],item['section']] for item in df]
    return(w)    

``` Что ещё можно достать нахаляву из df 
  {'audio': False,
  'color': '2',
  'flash': False,
  'id': '4070830',
  'is_breaking_news': False,
  'is_online': False,
  'live_text': False,
  'marked': False,
  'photos': False,
  'search_queries': None,
  'section': {'id': '25', 'title': 'Экономика и бизнес', 'url': '/ekonomika'},
  'show_at_common_feed': True,
  'show_at_section_feed': True,
  'slideshow': False
  'time': '1488624568',
  'title': 'Глава ВТБ: курс доллара к концу года может достигнуть 61-62 рублей',
  'topics': None,
  'url': '/ekonomika/4070830',
  'video': False}, ```

In [3]:
mainpage = "http://tass.ru"
def getPage(url):
    response = requests.get(mainpage+url)
                                       # выгружает данные по ссылке
    html = response.content            # переводит их в читаемый формат, который можно вывести на экран
    soup = BeautifulSoup(html,"lxml")  # отсекает кучу всяких ненужных вещей и запускает поиск по тегам в html  
    return soup

In [4]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) 
# очень крутые регулярные выражения - убирают все скобочки, тэги и прочую ерунду

def PageContent(url):
    a = getPage(url)
    vvv = a.findAll("div", { "class" : "b-material-text__l js-mediator-article" })
    text = vvv[0].findAll('p')
    textwp = [html_stripper(item) for item in text]
    return(textwp)

In [44]:
# Функция для объединения абзацев в текст
def Uniter(vect):
    a = vect[0]
    for item in vect[1:]:
        a = a + item
    return(a)

# Функция берёт одну конкретную новость и узнаёт про неё кучу полезной информации
def href_grab(unic_predata):
    url = unic_predata[2]
    
    try:
        precontent = PageContent(url)     # скачали контент страницы 
        count_abz = (len(precontent)) # посчитали количество абзацев
        text = Uniter(precontent)  # слили абзацы в текст
    except Exception:   # на Тасс обычно выскакивают ошибки на интервьюшках + ошибки из-за пустот (см. ниже)
        count_abz = 'Error' 
        text = 'Error'

    # Онормаливаем время
    if unic_predata[1] == None:
        new_time = 'Error'
    else:
        new_time = datetime.datetime.fromtimestamp(int(unic_predata[1])).strftime('%Y-%m-%d %H:%M:%S')
        # Отметим, что время будет записано в формате string

    # Блок для диагностики пустоты в куску с тематикой
    if unic_predata[4]==False:
        cur_id = 'Error'
        cur_title= 'Error'
    else:
        cur_id = unic_predata[4]['id']
        cur_title = unic_predata[4]['title']
    
    infa = [unic_predata[0],text,count_abz,new_time,unic_predata[2],unic_predata[3],cur_title,\
            cur_id]
    #[заголовок, текст, число абзацев, время публикации, ссылка, срочная ли новость, название раздела,
    # id раздела]
    return(infa)

Добавим немножечко параллельности в этот код. К сожелению, он будет состоять из двух кусков - линейный кусок, который листает вниз ленту новостей и собирает хрефы, заголовки и даты. Распаралеленный кусок берёт уже готовые хрефы и забирает по ним текст и прочую ерунду, которая находится непосредственно в самой по себе статье. Линейная часть делает 1 запрос на каждые 200 запросов нелинейной части. 

In [45]:
# Линейная часть
def Prevector_prepare(deep,start = 9999999999):
    time = start
    prevector = [ ]
    
    for i in range(deep):
        current = _lenta(time)
                
        # Дополнительный модуль по выявлению нехватающей даты
        i=1
        while current[-i][1]==None:
            i = i + 1
        prevector.extend(current)
        time=current[-i][1]
        #prevector.extend(current[:(-i+1)]) # Чтобы не возникало повторений (их и так не будет! Балбес!)
        #time = current[-1][1]
    return(prevector)
    
# вектор prevector включает в себя все хрефы на статьи, их заголовки и ещё кое-какой бред

In [46]:
# Функция Map для нелинейной части. Она берёт какой-то вектор с данными и скачивает всю инфу по нему
def Map(vect):
    wect = [href_grab(item) for item in vect]
    return(wect)

# Функция Reduce для нелинейной части. Она берёт все результаты разных Map и объединяет их в единый вектор
def Reduce(l):
    ll = [ ]
    for item in l:
        ll.extend(item)
    return(ll)

# Функция, которая делит входной поток данных на несколько частей
def Separator(vect,part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    prereduce = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(prereduce)

# Тестер того коректным ли было разбиение (на всякий случай)
def Test(begin,end):
    m=[]
    for item in end:
        m.extend(item)
    print(m==begin)

In [47]:
# Функция для выгрузки! Ура! Ура! Ура! 
def Kachatel(chto_kachat,chislo_potokov):
    # Map-шаг. В скобках у Pool задаётся количество потоков для работы 
    separatorlist = Separator(chto_kachat,chislo_potokov)
    Test(chto_kachat,separatorlist)
    if __name__ == '__main__':
        with Pool(chislo_potokov) as p:
            l = p.map(Map, separatorlist)

    # Reduce-шаг.
    itog = Reduce(l)
    return(itog)

In [9]:
# Погнали скачивать. Делай раз - линейная часть.
# Обязательный аргумент - глубина выкачки. Каждая дополнительная итерация это ещё 200 статей
# Качнём для начала 20 000 статей. 

start_time = time.time() # время работы кода 

prevector = Prevector_prepare(1000)

print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

--- 1920.7053484916687 seconds ---


Внимание! После Prevector_prepare(1,start=1471320873) будет ошибка, потому что prevector[-1] выдаёт

```['Все вологодские лагеря проверят после отравления детей под Череповцом',
 None,
 None,
 False,
 False]```

In [53]:
# Делай два - распаралеленная часть.
# Ставим в аргументы вектор из линейной части и число потоков
# Результаты тестирования времени и количества потоков: 
# 1000 статей 5 потоков по 200 - 219 секунд (около 4 минут)
# 2000 статей 10 потоков по 200 -221 секунда
# 2000 статей 20 потоков по 100 -160 секунд
# 20000 статей 20 потоков по 1000 штук - 1526 секунд (около 25 минут)
# 60000 cтатей 30 потоков              - 3467 секунд (2016-09-07)

test_vector = prevector

start_time = time.time() # время работы кода 

data_vector = Kachatel(test_vector,20)

print("--- %s seconds ---" % (time.time() - start_time))  #время работы кода

True
--- 15945.872259140015 seconds ---


In [57]:
prevector[-1]

['Документы о снятии с Ирана санкций опубликуют в США',
 '1445165027',
 '/mezhdunarodnaya-panorama/2356646',
 False,
 {'id': '22',
  'title': 'Международная панорама',
  'url': '/mezhdunarodnaya-panorama'}]

In [ ]:
1000*200

In [ ]:
1000000/20000*30/60

In [ ]:
1473255150

In [18]:
2015-10-18 13:43:47
1445165027

In [54]:
data = pd.DataFrame(data_vector)
data.columns = ['Заголовок','Текст','Количество абзацев','Дата','Ссылка','Срочность','Раздел','id Раздела']
data.tail(5)

,Заголовок,Текст,Количество абзацев,Дата,Ссылка,Срочность,Раздел,id Раздела
199994,"Болельщики ФК ""Спартак"" посвятили перформанс д...","МОСКВА, 18 октября. /Корр. ТАСС Олег Кошелев/....",4,2015-10-18 13:53:07,/sport/2356725,False,Спорт,2176
199995,Митинг в поддержку действий России в Сирии про...,"НЬЮ-ДЕЛИ, 18 октября. /Корр. ТАСС Евгения Карм...",12,2015-10-18 13:52:33,/mezhdunarodnaya-panorama/2356714,False,Международная панорама,22
199996,Сборная Украины по футболу планирует сыграть с...,"НЬОН /Швейцария/, 18 октября. /Спец. корр. ТАС...",8,2015-10-18 13:49:28,/sport/2356696,False,Спорт,2176
199997,Минобороны ДНР не фиксирует подготовку украинс...,"МОСКВА, 18 октября. /ТАСС/. Министерство оборо...",8,2015-10-18 13:48:48,/mezhdunarodnaya-panorama/2356678,False,Международная панорама,22
199998,Документы о снятии с Ирана санкций опубликуют ...,"ВАШИНГТОН, 18 октября. /Корр. ТАСС Антон Чудак...",14,2015-10-18 13:43:47,/mezhdunarodnaya-panorama/2356646,False,Международная панорама,22


In [55]:
# Сохранение и загрузка dataframe
data.to_csv('data.csv',sep=';',header=True)
# frame = pd.read_csv('dataset.tsv', header=0, sep='\t')

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 8 columns):
Заголовок             199999 non-null object
Текст                 199999 non-null object
Количество абзацев    199999 non-null object
Дата                  199999 non-null object
Ссылка                199997 non-null object
Срочность             199999 non-null bool
Раздел                199999 non-null object
id Раздела            199999 non-null object
dtypes: bool(1), object(7)
memory usage: 10.9+ MB


In [ ]:
np.mean(np.array(data['Абзацы']))

In [ ]:
data.dtypes

In [ ]:
#изменение типа столбца с помощью функции apply
data['Новое Время'] = data['Новое Время'].apply(pd.to_datetime)
data.dtypes

In [ ]:
ye = []
ye_mo = []
ye_mo_dd = []

for item in data['Новое Время']:
    ye.append(item.year)
    ye_mo.append(str(item.year)+'-'+str(item.month))
    ye_mo_dd.append(str(item.year)+'-'+str(item.month)+'-'+str(item.day))


In [ ]:
data['y']=ye
data['ym'] = ye_mo
data['ymd'] = ye_mo_dd
data.head(2)

In [ ]:
# Распределение новостей по годам. 
%pylab inline
data.y.value_counts().plot(kind='pie',figsize=(5,5))

In [ ]:
# Распределение новостей по месяцам. 
%pylab inline
data.ym.value_counts().plot(kind='pie',figsize=(5,5))  #'barh'

In [ ]:
# Распределение новостей по дням. 
%pylab inline
data.ymd.value_counts().plot(kind='barh')

In [ ]:
# подгружает разбиватель предложения на слова
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def Uniter(vect):
    a = vect[0]
    for item in vect[1:]:
        a = a + item
    return(a)

texts = [Uniter(item) for item in data['Текст']]
texts

In [ ]:
tokens = [tokenizer.tokenize(jtem) for jtem in texts]
tokens

In [ ]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('russian')

In [ ]:
stop

In [ ]:
out1 = [item for item in vect if not item in stop]

raw = text.lower()

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:

morph.parse(item)[0].normal_form for item in out1]


In [ ]:
morph.parse('люди')[0].normal_form

In [ ]:
morph.parse('люди')

In [ ]:
morph.parse('букотявку')